# 문제 유형3

* 다음은 영국에 위치한 온라인 쇼핑몰 U사의 2010년~2011년의 판매 데이터이며, 
  판매 영수증 정보는 다음과 같다.


    Online2.txt (구분자: tab, 541,909 Rows, 7 Columns, UTF-8 Encoding)

In [73]:
import pandas as pd
import numpy as np

import warnings

warnings.filterwarnings('ignore')

In [2]:
dat=pd.read_csv('Online2.txt', sep='\t', encoding='UTF-8')
dat.head(237)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
232,536390,22174,PHOTO CUBE,48,2010-12-01,1.48,17511.0,United Kingdom
233,536390,22969,HOMEMADE JAM SCENTED CANDLES,96,2010-12-01,1.25,17511.0,United Kingdom
234,536390,85099B,JUMBO BAG RED RETROSPOT,100,2010-12-01,1.65,17511.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01,1.65,17548.0,United Kingdom


In [3]:
dat.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

* 다음의 전처리를 먼저 수행한 후 분석 진행하시오.  

단계 1-1. 영수증 번호(InvoiceNo)가 결측치(Null, NA)인(= 수치형이 아닌 데이터) Row는 제거하시오.

In [4]:
print('원본 데이터 레코드 수: ', len(dat))

step1=dat[dat.InvoiceNo.str.isnumeric()]

print('영수증 번호(InvoiceNo) 결측치 제거 후 레코드 수: ', len(step1))

원본 데이터 레코드 수:  541909
영수증 번호(InvoiceNo) 결측치 제거 후 레코드 수:  532618


단계 1-2. 물품 단가(UnitPrice) 또는 구매 수량(Quantity)이 0 이하(<=0)인 Row는 제거하시오.


In [5]:
step2=step1[(step1.UnitPrice) > 0 & (step1.Quantity >0) ]

print('단계 1-2 처리 후 데이터 수: ', len(step2))

단계 1-2 처리 후 데이터 수:  530103


단계 1-3. 구매액(변수명: TotOrder) 변수를 추가하시오.  
 
         TotOrder = Quantity*UnitPrice


In [6]:
step3=step2.copy()

step3['TotOrder']=step3.Quantity*step3.UnitPrice

print('단계 1-3 처리 후 데이터 수: ', len(step3))

basetable1=step3.copy()

단계 1-3 처리 후 데이터 수:  530103


In [20]:
basetable1.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotOrder
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850.0,United Kingdom,20.34


* 상기 전처리를 완료한 데이터 프레임(데이터 프레임명: basetable1, 530,103 Rows)으로 다음 분석(문제 1~3) 수행

1. (basetable1을 이용) ‘영수증 번호(InvoiceNo)별 총 구매액(변수명: InvTotOrder)’의 평균이 국가별로 차이가 있는지 다음의 조건들을 참고하여 검정을 수행하고 이 때의 검정 통계량(F-value)을 기술하시오.  

- 하나의 InvoiceNo는 하나의 Country에 대응 
- InvTotOrder는 각 InvoiceNo에 대한 TotOrder의 합으로 정의 (19959개)
- 국가별 구매 건수(InvoiceNo 수)가 50이상(>=50) 이고 400이하(<=400)인 국가만을 대상으로 검정 수행, 자유도(df)=14
- 주어진 데이터는 등분산 조건을 만족한다고 가정
- 검정 통계량은 소수점 넷째 자리 이하는 버리고 셋째 자리까지만 기술  (답안 예시) 1.234


In [14]:
# 하나의 InvoiceNo는 하나의 Country에 대응 확인


tab1=pd.crosstab(basetable1.InvoiceNo, basetable1.Country)

tab1.index[(tab1 >= 1).sum(axis=1) > 1]

Index([], dtype='object', name='InvoiceNo')

In [32]:
Q1=pd.pivot_table(index=['InvoiceNo', 'Country'], values=['TotOrder'], 
                  data=basetable1, aggfunc='sum')
len(Q1)

19959

In [33]:
Q1.head(3)

,,TotOrder
InvoiceNo,Country,
536365,United Kingdom,139.12
536366,United Kingdom,22.20
536367,United Kingdom,278.73


In [34]:
Q1=Q1.reset_index()

In [35]:
Q1.columns.values[-1]='InvTotOrder'
print(Q1.columns)

Index(['InvoiceNo', 'Country', 'InvTotOrder'], dtype='object')


In [36]:
# 국가별 구매 건수(InvoiceNo 수)가 50이상(>=50) 이고 400이하(<=400)인 국가만을 대상으로 검정 수행, 자유도(df)=14

print(tab1.head(5))

country_list=tab1.columns[(tab1.sum() >= 50) & (tab1.sum() <= 400)]

print('대상 국가 목록: ', country_list.values)  # 자유도 14





Country    Australia  Austria  Bahrain  Belgium  Brazil  Canada  \
InvoiceNo                                                         
536365             0        0        0        0       0       0   
536366             0        0        0        0       0       0   
536367             0        0        0        0       0       0   
536368             0        0        0        0       0       0   
536369             0        0        0        0       0       0   

Country    Channel Islands  Cyprus  Czech Republic  Denmark  ...  RSA  \
InvoiceNo                                                    ...        
536365                   0       0               0        0  ...    0   
536366                   0       0               0        0  ...    0   
536367                   0       0               0        0  ...    0   
536368                   0       0               0        0  ...    0   
536369                   0       0               0        0  ...    0   

Country    Saudi A

In [38]:
Q1_1=Q1[Q1.Country.isin(country_list)]

Q1_1.Country.unique()

array(['Poland', 'Japan', 'Iceland', 'Denmark', 'Austria', 'Israel',
       'Greece', 'Hong Kong', 'Singapore', 'United Arab Emirates',
       'Canada', 'USA', 'European Community', 'Malta', 'RSA'],
      dtype=object)

In [40]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

Q1_2=ols('InvTotOrder~Country', Q1_1).fit()
Q1_out=anova_lm(Q1_2)
print(Q1_out)

Q1_ans=Q1_out['F'][0]

print('\n Q1 정답: ',  Q1_ans)  # 3.726266561864141

             df        sum_sq       mean_sq         F    PR(>F)
Country    14.0  6.414030e+07  4.581450e+06  3.726267  0.000035
Residual  120.0  1.475402e+08  1.229501e+06       NaN       NaN

 Q1 정답:  3.726266561864141


2. (basetable1을 이용)  ‘주문날짜(InvoiceDate)’와 ‘영수증 번호(InvoiceNo)별’ 총 구매액’(InvToOrder, 1번 참고) 테이블을 구성한 후 요일(Weekday)변수를 생성하시오. (InvoiceNo, InvTotOrder Weekday 컬럼을 가지는 데이터 프레임, 19959 rows).  
    이 때, 요일별 총 구매액이 상위 20%에 속하는 데이터만을 대상으로 총 구매액의 요일별 평균을 구하시오. 평균이 가장 낮은 요일과 평균 값을 기술하시오  
    - 평균 값은 정수 부분만 기술하시오. (답안예시) 일요일, 1


In [42]:
Q2=basetable1.pivot_table(index=['InvoiceDate', 'InvoiceNo'], values=['TotOrder'], 
                 aggfunc='sum').reset_index()

Q2['Weekday']=pd.to_datetime(Q2.InvoiceDate).dt.day_name(locale='ko_kr')

Q2.columns.values[2]='InvTotOrder'

print(Q2)

      InvoiceDate InvoiceNo  InvTotOrder Weekday
0      2010-12-01    536365       139.12     수요일
1      2010-12-01    536366        22.20     수요일
2      2010-12-01    536367       278.73     수요일
3      2010-12-01    536368        70.05     수요일
4      2010-12-01    536369        17.85     수요일
...           ...       ...          ...     ...
19954  2011-12-09    581583       124.60     금요일
19955  2011-12-09    581584       140.64     금요일
19956  2011-12-09    581585       329.05     금요일
19957  2011-12-09    581586       339.20     금요일
19958  2011-12-09    581587       249.45     금요일

[19959 rows x 4 columns]


In [45]:
Q2_2=Q2.groupby('Weekday').apply(lambda x: 
                                 x[x['InvTotOrder'] > x['InvTotOrder'].quantile(0.8)]).reset_index(drop=True)
print(Q2_2.head())


  InvoiceDate InvoiceNo  InvTotOrder Weekday
0  2010-12-03    536851      1368.40     금요일
1  2010-12-03    536856       754.87     금요일
2  2010-12-03    536862      1334.13     금요일
3  2010-12-03    536864       636.68     금요일
4  2010-12-03    536865      2396.96     금요일


In [51]:
Q2_out=Q2_2.groupby('Weekday')['InvTotOrder'].mean()

In [53]:
Q2_ans=[Q2_out.idxmin(), Q2_out[Q2_out.idxmin()]]

print('\n Q2 정답: ',  Q2_ans)  #['일요일', 942.2479365079362]


 Q2 정답:  ['일요일', 942.2479365079362]


3. (basetable1을 이용) 구매자(CustomerID)가 주문 날짜(InvoiceDate) 기준으로 동시에 구매한 물품(StockCode)들에 대한 연관성 분석을 다음 조건과 같이 수행하고자 합니다
- 분석 대상: 물품 단가(UnitPrice)의 평균이 4 이상(>=4)이고 구매(판매) 수량(Quantity)의 합이 600 이상(>=600)인 물품
- 구매자(CustomerID)와 주문 날짜(InvoiceDate)를 기준으로 물품 목록은 동일목록 존재 시 한 품목만 포함
- 구매자(CustomerID)와 주문 날짜(InvoiceDate)를 기준으로 한 종류의 물품만을 구매한 구매자는 제외
- 연관성 분석 시 구매자(CustomerID)와 주문 날짜(InvoiceDate)를 기준으로 물품목록을 추출한 후 연관성 분석(basket, transaction) 단위로 함
Note: 물품(StockCode)의 UnitPrice는 구매 시점마다 상이할 수 있습니다.
- 구매자 (CustomerID)를 확인할 수 없는 경우(결측치)는 분석에서 제외합니다.
- (Association Rule 모델 가이드) Min Support: 0.01, Min Confidence: 0.01, 그 외: Default

연관성 분석 결과 중, 선행(antecedent)과 후행(consequent)이 단일 아이템으로 구성된 rule만 추출하시오. 이 결과를 활용하여, 물품 '23012'을 주문한 고객에게 추천하기 적합한 품목을Lift 기준(내림차순)으로 선정하고, 그 때의 Lift를 기술하시오.
- Lift는 소수점 넷째 자리 이하는 버리고 셋째 자리까지만 기술하시오.
(답안예시) 1.234


In [60]:
Q3=basetable1.pivot_table(index='StockCode', 
                          values=['UnitPrice', 'Quantity'],
                          aggfunc={'UnitPrice': np.mean, 'Quantity': np.sum})

Q3.columns

Index(['Quantity', 'UnitPrice'], dtype='object')

In [64]:
# 물품 단가(UnitPrice)의 평균이 4 이상(>=4)이고 
# 구매(판매) 수량(Quantity)의 합이 600 이상(>=600)인 물품

filter_StockCode=Q3.index[(Q3.UnitPrice >= 4) & (Q3.Quantity >= 600)]
print(filter_StockCode)

Index(['15056BL', '15056N', '15056P', '20679', '20682', '20685', '20749',
       '20750', '20969', '20970',
       ...
       '84997C', '84997D', '85014A', '85014B', '85048', '85066', '85174',
       'DOT', 'M', 'POST'],
      dtype='object', name='StockCode', length=274)


In [66]:
Q3_1=basetable1[basetable1.StockCode.isin(filter_StockCode)]

In [68]:
# 구매자(CustomerID)와 주문 날짜(InvoiceDate)를 기준으로 
# 물품 목록은 동일목록 존재 시 한 품목만 포함

Q3_2=Q3_1.groupby(['CustomerID', 'InvoiceDate'])['StockCode'].apply(lambda x: x.unique())
print(Q3_2)

CustomerID  InvoiceDate
12347.0     2010-12-07     [71477, 22725, 22726, 22727, 22728, 22729, 849...
            2011-01-26     [22725, 22726, 22727, 22728, 22729, 21041, 210...
            2011-04-07                          [22371, 22550, 21041, 22423]
            2011-06-09            [23162, 23173, 23174, 22371, 22727, 22725]
            2011-08-02                                 [22371, 22727, 22423]
                                                 ...                        
18283.0     2011-09-05                                                   [M]
            2011-10-27                          [23494, 22960, 22625, 22550]
            2011-11-10                                               [22619]
18287.0     2011-05-22                                               [22752]
            2011-10-12                                        [21481, 22114]
Name: StockCode, Length: 14111, dtype: object


In [70]:
# 구매자(CustomerID)와 주문 날짜(InvoiceDate)를 기준으로 
# 한 종류의 물품만을 구매한 구매자는 제외

Q3_3=Q3_2[Q3_2.apply(len) > 1]

print(Q3_3)

CustomerID  InvoiceDate
12347.0     2010-12-07     [71477, 22725, 22726, 22727, 22728, 22729, 849...
            2011-01-26     [22725, 22726, 22727, 22728, 22729, 21041, 210...
            2011-04-07                          [22371, 22550, 21041, 22423]
            2011-06-09            [23162, 23173, 23174, 22371, 22727, 22725]
            2011-08-02                                 [22371, 22727, 22423]
                                                 ...                        
18283.0     2011-06-14                                        [23237, 23240]
            2011-06-23                       [84997A, 84997D, 84997B, 23298]
            2011-07-14                                 [22327, 23240, 23237]
            2011-10-27                          [23494, 22960, 22625, 22550]
18287.0     2011-10-12                                        [21481, 22114]
Name: StockCode, Length: 11682, dtype: object


In [74]:
# !pip install mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [98]:
te = TransactionEncoder()
te_ary = te.fit(Q3_3).transform(Q3_3)
df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df, min_support=0.01,
                            max_len=2, use_colnames=True)

Q3_rules=association_rules(frequent_itemsets, 
                           metric="confidence", min_threshold=0.01)

Q3_out=Q3_rules.sort_values(by='lift', ascending=False)
print(Q3_out)

Q3_ans=Q3_out[Q3_out.antecedents=={'23012'}][['consequents', 'lift']]

print('\n Q3 정답: \n ',  Q3_ans)
# (23013)  42.525773

    antecedents consequents  antecedent support  consequent support   support  \
177     (23012)     (23013)            0.016607            0.015152  0.010700   
176     (23013)     (23012)            0.015152            0.016607  0.010700   
116     (22634)     (22635)            0.017035            0.015323  0.010615   
117     (22635)     (22634)            0.015323            0.017035  0.010615   
179     (23127)     (23126)            0.014724            0.022770  0.011471   
..          ...         ...                 ...                 ...       ...   
82      (22423)     (22726)            0.134652            0.066684  0.011043   
65      (22423)     (22138)            0.134652            0.070536  0.011299   
64      (22138)     (22423)            0.070536            0.134652  0.011299   
96      (23298)     (22423)            0.082606            0.134652  0.010786   
97      (22423)     (23298)            0.134652            0.082606  0.010786   

     confidence       lift 